In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import division
import os
import numpy as np
import pandas as pd
from skimage.io import imread
import cv2
from pandas import read_csv
import matplotlib.pyplot as plt
import glob

%matplotlib inline

## Visualization

In [ ]:
from evaluate import load_network
import cPickle as pickle
from models import *
from utils import get_file_list, create_fixed_image_shape
from nolearn.lasagne import BatchIterator
from plotting import plot_face_bb
# from lazy_batch_iterator import LazyBatchIterator
import skimage
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
train_folder = 'train/'
fnames, bboxes = get_file_list(train_folder)

In [ ]:
def prepare_test_img(shape, img=None, name=None):
    from skimage.io import imread 
    im = None
    if img is not None:
        im = img
    elif name is not None:
        im = imread(name)
    inp = create_fixed_image_shape(im, shape, mode='fit')
    return inp

### Option 1 - Load weights

In [ ]:
nnet = load_network("/home/arya_03/work/face_detection_gtx/model_1234_sigmoid_trans_scale/model_19.pkl",
                    "nnet_4c3d_1234_convs_layer",
                    "BatchIterator")

### Option 2 - Full network

In [ ]:
full_model_name = '/home/arya_03/work/face_detection_gtx/model_1234_sigmoid_trans_scale/full_nnet.pkl'
nnet = pickle.load(open(full_model_name, 'rb'))
nnet.batch_iterator_test = BatchIterator(batch_size=64)

### Evaluate result on the test images

In [ ]:
# Predict and plot the bounding boxes for images in test_imgs/ folder
for fname in glob.glob(os.path.join('test_imgs', '*.jpg')):
    proc_img = prepare_test_img((256, 256, 3), name=fname)
    img = np.transpose(proc_img, [2, 0, 1])
    preds = nnet.predict([img])
    plot_face_bb(proc_img, preds[0], path=False)

### View augmented images

In [ ]:
for fname in glob.glob('*.jpg'):
    plt.figure()
    plt.imshow(imread(fname))

## Plot x-y-scale for the faces used for training

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
df = read_csv('aug.csv')
df.head()

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df['r'][:4000], df['c'][:4000], df['width'][:4000], c='none', facecolors='none', edgecolors='r')

In [ ]:
df.describe()

In [ ]:
%matplotlib inline
plt.hist2d(df['r'], df['c'], bins=30);